# Getting Object and Boundaries from Images

##### INSTALLED

In [2]:
# pip install --upgrade google-cloud-vision
#pip install opencv-python

In [3]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

#import OpenCV to annotate images with shapes
import cv2

# google cloud SDK
import io
import os

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

In [4]:
# Code Reference From: https://github.com/googleapis/google-cloud-python/issues/5349
# need this in order to access API - enter your JSON file in the path after you have created your service account

from google.oauth2 import service_account
credentials = service_account.Credentials. from_service_account_file("C:/Users/Erik/Desktop/flood_project/flooddepths-0158d30f7869.json")

### List of Images

In [5]:
# a list of all of my picture filenames that have floods & cars
# with help from https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory

flood_list = [f for f in os.listdir("C:/Users/Erik/Desktop/flood_project/images/flood/") if os.path.isfile(os.path.join("C:/Users/Erik/Desktop/flood_project/images/flood/", f))]

In [6]:
# a list of all of my picture filenames that have cars (no floods)
no_flood_list = [f for f in os.listdir("C:/Users/Erik/Desktop/flood_project/images/no_flood/") if os.path.isfile(os.path.join("C:/Users/Erik/Desktop/flood_project/images/no_flood/", f))]

In [7]:
flood_list

['flooded_ (1).jpg',
 'flooded_ (10).jpg',
 'flooded_ (100).jpg',
 'flooded_ (101).jpg',
 'flooded_ (102).jpg',
 'flooded_ (103).jpg',
 'flooded_ (104).jpg',
 'flooded_ (105).jpg',
 'flooded_ (106).jpg',
 'flooded_ (107).jpg',
 'flooded_ (108).jpg',
 'flooded_ (109).jpg',
 'flooded_ (11).jpg',
 'flooded_ (12).jpg',
 'flooded_ (13).jpg',
 'flooded_ (14).jpg',
 'flooded_ (15).jpg',
 'flooded_ (16).jpg',
 'flooded_ (17).jpg',
 'flooded_ (18).jpg',
 'flooded_ (19).jpg',
 'flooded_ (2).jpg',
 'flooded_ (20).jpg',
 'flooded_ (21).jpg',
 'flooded_ (22).jpg',
 'flooded_ (23).jpg',
 'flooded_ (24).jpg',
 'flooded_ (25).jpg',
 'flooded_ (26).jpg',
 'flooded_ (27).jpg',
 'flooded_ (28).jpg',
 'flooded_ (29).jpg',
 'flooded_ (3).jpg',
 'flooded_ (30).jpg',
 'flooded_ (31).jpg',
 'flooded_ (32).jpg',
 'flooded_ (33).jpg',
 'flooded_ (34).jpg',
 'flooded_ (35).jpg',
 'flooded_ (36).jpg',
 'flooded_ (37).jpg',
 'flooded_ (38).jpg',
 'flooded_ (39).jpg',
 'flooded_ (4).jpg',
 'flooded_ (40).jpg',
 'fl

In [8]:
#Test set of images
test_flood_images = ['flooded_ (11).jpg','flooded_ (26).jpg']

In [9]:
objects_to_crop_around = ['Car','Van','Truck','Boat','Toy vehicle']

## Function to get the objects and their boundaries

In [12]:
# Just gets the objects and their boundaries.
def determine_objects_boundary(file_list, string__no_flood_OR_flood):
    
    file_dict = {}
    object_dict = {}
    
    #looping though each image in the list submitted to the function
    for file in file_list:
    
        #need to have google vision credentials saved to credentials
        client = vision.ImageAnnotatorClient(credentials=credentials)

        # path to the images that need to be cropped
        with open('C:/Users/Erik/Desktop/flood_project/images/' + string__no_flood_OR_flood + '/' + file, 'rb') as image_file:
            content = image_file.read()
        image = vision.types.Image(content=content)
        
        #same path just using OpenCV to get image shape
        im_cv2 = cv2.imread('C:/Users/Erik/Desktop/flood_project/images/' + string__no_flood_OR_flood + '/' + file)
        height, width, color = im_cv2.shape

        #Using Google vision to actually find objects in the image
        objects = client.object_localization(image=image).localized_object_annotations
        
        object_list = []
        
        #for each object in the image, find the 4 corners of the bounding rectangle and save them in a dict
        for object_ in objects:
            vertex_dict = {}

            #need to make sure the normalized vertex are multipled by the corresponding image distance so the vertex are in pixels counts
            for index,vertex in enumerate(object_.bounding_poly.normalized_vertices):
                vertex_dict[f'vertex_{index}'] = [int(width*vertex.x),int(height*vertex.y)]
            object_dict[object_.name] = vertex_dict
            
        file_dict[file] = object_dict
            
    return file_dict

In [14]:
test_image = determine_objects_boundary(test_flood_images, 'flood')

In [15]:
test_image['flooded_ (11).jpg']

{'Truck': {'vertex_0': [247, 111],
  'vertex_1': [421, 111],
  'vertex_2': [421, 165],
  'vertex_3': [247, 165]},
 'Car': {'vertex_0': [33, 163],
  'vertex_1': [312, 163],
  'vertex_2': [312, 254],
  'vertex_3': [33, 254]}}

## Cropping images to use in NN

In [42]:
def crop_flooded_objects_boundary(file_list):
    
    file_dict = {}
    object_dict = {}
    
    # creating a tenth and cycle counter to output progress of function
    tenth_counter = 0
    cycle_counter = 1
    
    #looping though each image in the list submitted to the function
    for file in file_list:
    
        #need to have google vision credentials saved to credentials
        client = vision.ImageAnnotatorClient(credentials=credentials)

        # path to the images that need to be cropped
        with open('C:/Users/Erik/Desktop/flood_project/images/flood/' + file, 'rb') as image_file:
            content = image_file.read()
        image = vision.types.Image(content=content)
        
        #same path just using OpenCV to get image shape and will use to save the cropped images later
        im_cv2 = cv2.imread('C:/Users/Erik/Desktop/flood_project/images/flood/' + file)
        height, width, color = im_cv2.shape

        #Using Google vision to actually find objects in the image
        objects = client.object_localization(image=image).localized_object_annotations
        
        object_list = []
        
        tenth_counter += 1
        
        # printing out the fraction of the images done to keep track of function progress
        if int(round(len(flood_list),-1)/10) <= tenth_counter:
            print(f'{cycle_counter}/10 done')
            cycle_counter += 1
            tenth_counter = 1
        
        # creating an item counter to allow the cropping and the saving of multiple same objects from one photo
        item_counter = 1
        
        #looping through each of the objects Google vision found in the image
        for object_ in objects:
            # ignoring all objects that don't have to do with the cars in the image
            if object_.name in objects_to_crop_around:
                vertex_dict = {}

                #need to make sure the normalized vertex are multipled by the corresponding image distance so the vertex are in pixels counts
                for index,vertex in enumerate(object_.bounding_poly.normalized_vertices):
                    vertex_dict[f'vertex_{index}'] = [int(width*vertex.x),int(height*vertex.y)]
                object_dict[object_.name] = vertex_dict
            
                # Cropping the image around the vertices of the object
                
                # https://www.life2coding.com/cropping-polygon-or-non-rectangular-region-from-image-using-opencv-python/
                # https://stackoverflow.com/questions/48301186/cropping-concave-polygon-from-image-using-opencv-python
                
                mask = np.zeros(im_cv2.shape[:2], np.uint8)
                points = np.array([object_dict[object_.name]['vertex_0'],
                                   object_dict[object_.name]['vertex_1'],
                                   object_dict[object_.name]['vertex_2'],
                                   object_dict[object_.name]['vertex_3']])
            
                #creating the bounding rectangle from the object vertices
                rect = cv2.boundingRect(points)
                x,y,w,h = rect
                
                # cropping the image using OpenCV and the dimentions of the bounding rectangle
                cropped = im_cv2[y:y+h, x:x+w].copy()
            
                #savig the cropped image using OpenCV. Image name has cropped_(object)_ added to the front
                cv2.imwrite('C:/Users/Erik/Desktop/flood_project/images/cropped_flood/cropped_' + object_.name + str(item_counter) + '_' + file, cropped)
                
        file_dict[file] = object_dict
        
    return file_dict

In [27]:
#crop_flooded_objects_boundary(test_flood_images);

In [58]:
#cropping all imaged in the flood_list. Run with a semi colon to suppress the dictionary from showing upon completion
crop_flooded_objects_boundary(flood_list);

1/10 done
